<a href="https://colab.research.google.com/github/sayarghoshroy/wikisearch/blob/master/index_maker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import time
start = time.time();

import nltk
nltk.download('punkt')
import xml.sax
import subprocess
from copy import copy

import pickle
import sys
import re
import os
import shutil
# from nltk.tokenize.regexp import regexp_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from operator import itemgetter

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
stemmed_dict = {}

def stem(token):
  if token in stemmed_dict:
    return stemmed_dict[token]
  temp = snowball_stemmer.stem(token)
  stemmed_dict[token] = temp
  return temp

In [ ]:
nltk.download('stopwords')

stopword = stopwords.words('english')
snowball_stemmer = SnowballStemmer('english')
word_set = {}

indexed_dict = {}
doc_id = 0

token_count = 0
for word in stopword:
  word_set[word] = None

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
def clean(txt):
  txt = txt.replace("\n", " ").replace("\r", " ")
  punc_list = '!"#$&*+,-./;?@\^_~)('
  t = str.maketrans(dict.fromkeys(punc_list, " "))
  txt = txt.translate(t)
  t = str.maketrans(dict.fromkeys("'`", ""))
  txt = txt.translate(t)

  return txt

def regtok(txt):
  txt = clean(txt)
  regex = re.compile(r'(\d+|\s+|=|}}|\|)')
  tokens = [stem(token) for token in regex.split(txt) if token not in word_set and (token.isalnum() or token == '}}' or token == '{{infobox')]
  # tokens = regexp_tokenize(txt, pattern = '\s+', gaps = True)
  return tokens

In [ ]:
# Example to show tokenization
regtok("hello my name is {{Infobox a=100| anarchism|b=10| (hi) c = 900 | 2020 1st | }} 25th 2020 Sayar. Externallinks what is your name?")

['hello',
 'name',
 '100',
 'anarch',
 'b',
 '10',
 'hi',
 'c',
 '900',
 '2020',
 '1',
 'st',
 '}}',
 '25',
 'th',
 '2020',
 'sayar',
 'externallink',
 'name']

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# Template: https://towardsdatascience.com/wikipedia-data-science-working-with-the-worlds-largest-encyclopedia-c08efbac5f5c

class WikiXmlHandler(xml.sax.handler.ContentHandler):
    """Content handler for Wiki XML data using SAX"""
    def __init__(self):
        xml.sax.handler.ContentHandler.__init__(self)
        self._buffer = None
        self._values = {}
        self._current_tag = None
        self._pages = []

    def characters(self, content):
        """Characters between opening and closing tags"""
        if self._current_tag:
            self._buffer.append(content)

    def startElement(self, name, attrs):
        """Opening tag of element"""
        if name in ('title', 'text'):
            self._current_tag = name
            self._buffer = []

    def endElement(self, name):
        """Closing tag of element"""
        if name == self._current_tag:
            self._values[name] = ' '.join(self._buffer)

        if name == 'page':
            self._pages.append((self._values['title'], self._values['text']))

In [ ]:
# title: 0, infobox: 1, body: 2, categories: 3, references: 4, external_links: 5

class NewHandler(xml.sax.ContentHandler):
	def __init__(self):
		xml.sax.ContentHandler.__init__(self)
		self.title_flag = False
		self.text_flag = False
		self.info_flag = False
		self.text_string = ""

	def startElement(self, name, attrs):
		global doc_id
		if name == "title":
			self.title_flag = True
		elif name == "text":
			self.text_flag = True
		elif name == "page":
			doc_id = doc_id + 1

	def endElement(self, name):
		global doc_id
		global indexed_dict
		global token_count

		info_tokens = []
		body_tokens = []
		link_tokens = []
		category_str = ""
		refer_str = ""
		body_flag = True
		link_flag = False
		info_flag = False

		if name == "text":
			self.text_flag = False
			tokens = regtok(self.text_string)
			
			categories_str = re.findall('(?<=\[\[category:)(.*?)(?=\]\])', self.text_string)
			ref_type_1 = re.findall('(?<=\* \[\[)(.*?)(?=\])', self.text_string)
			ref_type_2 = re.findall('(?<=\* \{\{)(.*?)(?=\}\})', self.text_string)

			category_tokens = []
			for stri in categories_str:
				all_tok = regtok(stri)
				for tok in all_tok:
					token_count += 1
					category_tokens.append(tok)

			refer_tokens = []
			for stri in ref_type_1:
				all_tok = regtok(stri)
				for tok in all_tok:
					token_count += 1	
					refer_tokens.append(tok)

			for stri in ref_type_2:
				all_tok = regtok(stri)
				for tok in all_tok:
					token_count += 1
					refer_tokens.append(tok)

			for token in tokens:
				token_count += 1
				if token == '{{infobox':
					info_flag = True
					body_flag = False
					continue

				if token == 'externallink':
					link_flag = True
					body_flag = False
					continue
				
				if info_flag is False and link_flag is False:
					body_flag = True
				
				if info_flag is True:
					if token == '}}':
						info_flag = False
						continue
					info_tokens.append(token)

				elif body_flag is True:
					body_tokens.append(token)
		 
				elif link_flag is True and info_flag is False:
					link_tokens.append(token)

			# Body
			for key in refer_tokens:
				if key not in indexed_dict:
					indexed_dict[key] = [[], [], [], [], [], [], [0, 0, 0, 0, 0, 0]]
				
				indexed_dict[key][6][4] += 1
				if not indexed_dict[key][4] or indexed_dict[key][4][-1] != doc_id:
					indexed_dict[key][4].append(doc_id)

			for key in category_tokens:
				if key not in indexed_dict:
					indexed_dict[key] = [[], [], [], [], [], [], [0, 0, 0, 0, 0, 0]]
				
				indexed_dict[key][6][3] += 1
				if not indexed_dict[key][3] or indexed_dict[key][3][-1] != doc_id:
					indexed_dict[key][3].append(doc_id)

			for key in body_tokens:
				if key not in indexed_dict:
					indexed_dict[key] = [[], [], [], [], [], [], [0, 0, 0, 0, 0, 0]]
				
				indexed_dict[key][6][2] += 1
				if not indexed_dict[key][2] or indexed_dict[key][2][-1] != doc_id:
					indexed_dict[key][2].append(doc_id)

			for key in link_tokens:
				if key not in indexed_dict:
					indexed_dict[key] = [[], [], [], [], [], [], [0, 0, 0, 0, 0, 0]]
				
				indexed_dict[key][6][5] += 1
				if not indexed_dict[key][5] or indexed_dict[key][5][-1] != doc_id:
					indexed_dict[key][5].append(doc_id)

			# Infobox

			for key in info_tokens:
				if key not in indexed_dict:
					indexed_dict[key] = [[], [], [], [], [], [], [0, 0, 0, 0, 0, 0]]
				
				indexed_dict[key][6][1] += 1
				if not indexed_dict[key][1] or indexed_dict[key][1][-1] != doc_id:
					indexed_dict[key][1].append(doc_id)

			self.text_string = ""

	def characters(self, data):
		global doc_id
		global indexed_dict
		global token_count

		if self.title_flag is True:
			data = data.replace(":", " ")
			tokens = regtok(data)
			
			for key in tokens:
				token_count += 1
				if key not in indexed_dict:
					indexed_dict[key] = [[], [], [], [], [], [], [0, 0, 0, 0, 0, 0]]
				
				indexed_dict[key][6][0] += 1
				if not indexed_dict[key][0] or indexed_dict[key][0][-1] != doc_id:
					indexed_dict[key][0].append(doc_id)
			
			self.title_flag = False

		if self.text_flag is True:
			self.text_string = self.text_string + " " + data.lower().replace("external links", "externallinks")

In [ ]:
data_path_zipped = 'gdrive/My Drive/ph1.xml-p1p30303.bz2'
# !bzip2 -d 'gdrive/My Drive/ph1.xml-p1p30303.bz2'
data_path_unzipped = 'gdrive/My Drive/ph1.xml-p1p30303'

In [ ]:
cutoff = int(1e9)
handler = NewHandler()
parser = xml.sax.make_parser()
parser.setContentHandler(handler)

# For Running on the Zipped Version of the Dump:
# for line in subprocess.Popen(['bzcat'], stdin = open(data_path), stdout = subprocess.PIPE).stdout:
#     parser.feed(line)

#     if doc_id > cutoff:
#       break

# For Running on the UnZipped Dump:
# xml.sax.parse(data_path_unzipped, handler)

# file_handle = open(data_path_unzipped, 'r+')
# start_load = time.time()
# lines = file_handle.readlines()
# end_load = time.time()
# size = len(lines)

line = -1
doc_id = 0
count = 0

# while count < size:
with open(data_path_unzipped, "r+") as file_handle:
  while line != '':
      # Get next line from file 
      # line = lines[count]
      line = file_handle.readline()
      parser.feed(line)
      if doc_id > cutoff:
        break
      count += 1

In [ ]:
pickle_out = open("index.pkl", "wb")
pickle.dump(indexed_dict, pickle_out)

In [ ]:
end = time.time()
print("Number of Docs Processed =", doc_id)
print("Time Taken = " + str((end - start) / 60) + " minutes")
print("Token Count: " + str(token_count))
# print("Load Time = " + str((end_load - start_load) / 60) + " minutes")
cnt = 0
for key in indexed_dict.keys():
  cnt += 1
print("Number of Keys: " + str(cnt))

Number of Docs Processed = 19797
Time Taken = 8.49585642417272 minutes
Token Count: 70611769
Number of Keys: 1329158


In [ ]:
cnt = 0
for key in indexed_dict.keys():
  cnt += 1
  if cnt < 10:
    print(str(key) + ": " + str(indexed_dict[key][6]))

accessiblecomput: [1, 0, 0, 0, 0, 0]
redirect: [2, 1, 5932, 37, 12, 37]
move: [2, 11, 23173, 8, 73, 72]
}}: [0, 0, 1311635, 4, 53, 91082]
camelcas: [0, 0, 303, 0, 0, 4]
unprintworthi: [0, 0, 60, 20, 0, 0]
anarch: [3, 0, 1212, 14, 183, 10]
countri: [26, 1422, 64962, 1133, 805, 1388]
govern: [41, 192, 54923, 285, 508, 858]


In [ ]:
pickle_in = open("index.pkl", "rb")
index = pickle.load(pickle_in)

In [ ]:
# title: 0, infobox: 1, body: 2, categories: 3, references: 4, external_links: 5

def field_query(query):
  # t:World Cup i:2019 c:Cricket
  toks = regtok(query.lower().replace("t:", " ").replace("i:", " ").replace("r:", " ").replace("l:", " ").replace("b:", " ").replace("c:", " "))
  tokens = [stem(word) for word in toks if word not in word_set]

  print()

  for token in tokens:
    if token not in index:
      print("Token: " + str(token) + " NOT FOUND")
      continue
    
    print("Stem: " + str(token))
    print("Counts:")
    print("Title: " + str(index[token][6][0]) + ", Infobox: " + str(index[token][6][1]) + ", Categories: " + str(index[token][6][3]) + ", References: " + str(index[token][6][4]) + ", Body: " + str(index[token][6][2]) + ", Links: " + str(index[token][6][5]))
    print("Postings:")
    print("Title: " + stri(index[token][0]))
    print("Infobox: " + stri(index[token][1]))
    print("Categories: " + stri(index[token][3]))
    print("References: " + stri(index[token][4]))
    print("Body: " + stri(index[token][2]))
    print("Links: " + stri(index[token][5]))
    print()

  return

field_query("t:World Cricket i:1997 c:2020 l:25th")
# field_query("t: refend")

Stem: world
Counts:
Title: 30, Infobox: 439, Categories: 968, References: 3292, Body: 103718, Links: 3010
Postings:
Title: [1166, 2229, 2230, 2984, 3060, 3213, 3617, 4418, 5864, 5866, 5968, 6411, 7151, 7318, 7763, 7774, 8792, 9068, 9442, 10443, 10464, 12386, 14046, 14829, 16376, 16648, 16860, 17070, 18367, 19482]
Infobox: [102, 104, 105, 171, 173, 270, 329, 342, 396, 406, 408, 440, 442, 446, 548, 632, 674, 954, 1007, 1088, 1346, 1358, 1462, 1574, 1708, 1993, 2040, 2042, 2049, 2057, 2091, 2096, 2103, 2106, 2111, 2114, 2210, 2219, 2229, 2230, 2263, 2269, 2351, 2434, 2472, 2516, 2517, 2657, 2672, 2696, 2756, 2766, 2790, 2810, 2813, 2886, 2984, 3060, 3081, 3240, 3257, 3284, 3293, 3341, 3350, 3356, 3361, 3368, 3435, 3530, 3539, 3610, 3719, 4397, 4418, 4726, 4793, 4860, 4931, 4938, 4944, 4961, 5056, 5346, 5650, 5782, 5791, 5798, 5800, 5810, 5815, 5827, 5833, 5966, 6035, 6060, 6098, 6134, 6164, 6171, 6339, 6629, 6717, 6731, 6837, 6851, 6958, 6959, 7066, 7107, 7151, 7485, 7513, 7579, 7582, 759

In [ ]:
# ^_^ Thank You